# Define figure projection
Here we should the most minimal use of `PolarToolkit`. This example just creates a projection in EPSG:3031, based on a region and figure height (or width). The rest of the example uses standard PyGMT calls.

Import the packages

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2

# set Python's logging level to get information
import logging

import pygmt

from polartoolkit import fetch, maps, utils

logging.getLogger().setLevel(logging.INFO)

Define a region for the plot

In [ ]:
# Options:

# 1) use the full extent of the grid file
# region = utils.get_grid_info(bed)[1]

# 2) use a preset region (polartoolkitgions())
# region = regions.antarctic_peninsula

# 3) define your own region, in meters e, w, n, s in EPSG 3031 (for Antarctica) or
# EPSG 3413 (for the Arctic)

region = (-2700e3, -2000e3, 1000e3, 2000e3)

print(f"region: {region}")

Fetch the data to plot

In [ ]:
bed = fetch.bedmap2(layer="bed", region=region)
bed

Create a projection from the region and a figure height

In [ ]:
proj_xy = utils.set_proj(region, hemisphere="south", fig_height=15)[0]

Use standard PyGMT commands to plot a figure

In [ ]:
fig = pygmt.Figure()

fig.grdimage(
    grid=bed,
    cmap="globe",
    projection=proj_xy,
    region=region,
    frame=True,
    nan_transparent=True,
)

# display the figure
fig.show()

In [ ]:
# display colorbar 2/3 as wide as figure

fig.colorbar(
    cmap=True,
    position=f"jBC+w{utils.get_fig_width()*(2/3)}c/.5c+jTC+h+o0c/.6c+e",
    frame="xaf+lBedmap2 bed (m)",
)

fig.show()

You can also create the projection by giving a figure width instead of height

In [ ]:
proj_xy = utils.set_proj(region, hemisphere="south", fig_width=15)[0]

# use standard PyGMT commands to plot figure
fig = pygmt.Figure()

# create a custom colarmap
pygmt.makecpt(
    cmap="globe",
    series="-4500/2500/250",  # 250m increments between -4.5 and +2.5 km.
)

fig.grdimage(
    grid=bed,
    cmap=True,
    projection=proj_xy,
    region=region,
    frame=True,
    nan_transparent=True,
)

# display the figure
fig.show()

You can switch between standard PyGMT commands and PolarToolkit commands. 
Here, on the same figure instance, we'll add:
* a colorbar with a histogram
* an inset location map
* the Antarctic coastline and groundingline

In [ ]:
maps.add_coast(fig, pen="1p,red", hemisphere="south")

maps.add_inset(fig, inset_width=0.4, inset_box_pen="2p,green", hemisphere="south")

maps.add_colorbar(
    fig,
    cbar_label="Bedmap2 bed elevation",
    cbar_unit="m",
    hist=True,
    cpt_lims=(-4500, 2500),
    grid=bed,
    region=region,
    hist_bin_width=250,  # set this to the cmap interval to match hist bins to cmap bins
    # hist_bin_num=20, # use this instead to set the number of bins
)

fig.show()